# Matrix Factorization 

## Setup
I am using cython because it allows me to speed up the training of the model

### Cython -- WIP

In [6]:
%load_ext Cython
%%cython

UsageError: Line magic function `%%cython` not found.


### Imports

In [2]:
%cd ../

c:\Progetto_Ing_Informatica\Master\RecSys


In [3]:
import time
import numpy as np
import pandas as pd
from numpy.ma import MaskedArray
import sklearn.utils.fixes
import scipy.sparse as sps
from functions import split_train_in_three_percetanges

## Data

### Paths

In [4]:
URM_PATH= 'Data/interactions_and_impressions.csv'

### Load data

In [23]:
# Load the data
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_PATH,
                                sep=",",
                                header=0, engine='python')
URM_all_dataframe.columns = ["UserID", "ItemID", "Impressions", "Data"]

print("The number of interactions is {}".format(len(URM_all_dataframe)))

The number of interactions is 5826506


In [6]:
URM_all_dataframe.head(5)

,UserID,ItemID,Impressions,Data
0,0,11,"0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19",1
1,0,21,NaN,0
2,0,21,NaN,0
3,0,21,"20,21,22,23,24,25,26,27,28,29",0
4,0,21,NaN,1


In [7]:
userID_unique = URM_all_dataframe["UserID"].unique()
itemID_unique = URM_all_dataframe["ItemID"].unique()

### Costants

In [8]:
n_users = len(userID_unique)
n_items = len(itemID_unique)
n_iteractions = len(URM_all_dataframe)

### Analyse

In [9]:
print("Number of items\t {}, Number of users\t {}".format(n_items, n_users))
print("Max ID items\t {}, Max Id users\t {}\n".format(max(itemID_unique), max(userID_unique)))
print("Number of interactions\t {}".format(n_iteractions))

Number of items	 24507, Number of users	 41629
Max ID items	 24506, Max Id users	 41628

Number of interactions	 5826506


In [10]:
print("Average interactions per user {:.2f}".format(n_iteractions / n_users))
print("Average interactions per item {:.2f}\n".format(n_iteractions / n_items))

Average interactions per user 139.96
Average interactions per item 237.75



In [11]:
print("Sparsity {:.2f} %".format((1 - float((n_iteractions) / (n_items * n_users))) * 100))

Sparsity 99.43 %


## Paramenters

In [28]:
VALIDATION_RATIO = 0.15
TEST_RATIO = 0.1

NUM_LATENT_FACTORS = 10

WISE_USER = False # If True, select the number of interactions one user at a time. Otherwise, globally.

## Implict URM

In [24]:
#Create a binary matrix with one per each interaction
URM_all = sps.coo_matrix((np.ones(len(URM_all_dataframe["Data"].values)),
                          (URM_all_dataframe["UserID"].values, URM_all_dataframe["ItemID"].values)))
URM_all = URM_all.tocsr()  # to obtain fast access to rows (users)

URM_train, URM_validation, URM_test = split_train_in_three_percetanges(URM_all, WISE_USER, VALIDATION_RATIO, TEST_RATIO)

## Funk SVD

### Creation of the random matrices

In [29]:
#Create random matrices
user_factors = np.random.random((n_users, NUM_LATENT_FACTORS))
item_factors = np.random.random((n_items, NUM_LATENT_FACTORS))

In [30]:
URM_train_coo = URM_train.tocoo()

sample_index = np.random.randint(URM_train_coo.nnz)
sample_index

212730

In [31]:
user_id = URM_train_coo.row[sample_index]
item_id = URM_train_coo.col[sample_index]
rating = URM_train_coo.data[sample_index]

(user_id, item_id, rating)

(7122, 23153, 1.0)

### Sample of a prediction
In this case is completely random due to the lack of training

In [32]:
#Remember that the first prediction is just random
predicted_rating = np.dot(user_factors[user_id,:], item_factors[item_id,:])
predicted_rating

3.5143175035751995

In [33]:
prediction_error = rating - predicted_rating
prediction_error

-2.5143175035751995